In [59]:
import pandas as pd
import numpy as np

!pip install catboost >> none
import catboost
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score

import warnings
warnings.filterwarnings('ignore')

In [76]:
df = pd.read_csv('train_dataset_train (1).csv')

In [77]:
pd.set_option('display.max_columns', None)
df

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,mailtype,mailctg,mailrank,directctg,transport_pay,postmark,name_mfi,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
0,6818780,1043_-1,628629.0,Участок,7503.0,N,0.0,Y,N,87.0,5.0,1.0,0.0,2.0,0.00,0.0,"Fishhook,USB",41.0,150.0,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,0
1,9907176,1023_-1,102976.0,ММПО,7503.0,N,0.0,N,N,107.0,5.0,1.0,0.0,2.0,0.00,0.0,"screen protector,Case(Q613B),case(208B40-DB)",68.0,400.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0
2,3304275,1018_-1,620962.0,Цех,7503.0,N,0.0,Y,N,50.0,5.0,1.0,0.0,2.0,0.00,0.0,"Pendant Necklaces,Rings for Women,Necklaces",56.0,218.0,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,0
3,9020937,1019_-1,344964.0,Цех,7503.0,N,0.0,Y,N,416.0,5.0,1.0,0.0,2.0,35.34,0.0,Motorcycle Signal Lamp,33.0,100.0,55.0,2060928.0,653280.0,2714208.0,19562334.0,0,0,0,0,0
4,3082311,1020_-1,629819.0,Участок,7503.0,N,0.0,Y,N,795.0,5.0,1.0,0.0,2.0,52.52,0.0,backpack,716.0,1000.0,16.0,316919.0,27911.0,344830.0,4719186.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,9958614,1022_-1,102976.0,ММПО,7503.0,N,0.0,N,N,25.0,5.0,0.0,0.0,2.0,0.00,0.0,KEY CHAIN,24.0,100.0,1089.0,64270133.0,116432632.0,180702765.0,188407812.0,0,0,1,0,0
5999996,2234489,1022_-1,241963.0,Цех,7503.0,N,0.0,Y,N,83.0,5.0,1.0,0.0,2.0,0.00,0.0,Dolls,100.0,1832.0,31.0,1767370.0,144063.0,1911433.0,15582018.0,0,0,0,0,0
5999997,4304572,1041_-1,102971.0,Цех,7506.0,N,0.0,N,N,1700.0,5.0,1.0,0.0,2.0,94.09,0.0,Down jacket,952.0,800.0,186.0,60613352.0,10648.0,60624000.0,75592387.0,0,0,0,0,0
5999998,6550634,1018_-1,102152.0,Цех,7506.0,N,0.0,N,N,269.0,5.0,1.0,0.0,2.0,0.00,0.0,0,0.0,0.0,105.0,15091338.0,4972424.0,20063762.0,39988530.0,0,1,0,0,0


In [78]:
test = pd.read_csv('test_dataset_test.csv')

In [79]:
df.index_oper = df.index_oper.astype(str)

dfstr = df.loc[df.index_oper.str.slice(-2,) == '.0']
dfstr1 = df.loc[(df.index_oper.str.slice(-2,) != '.0')]

dfstr1.index_oper.replace(' ', np.nan, inplace=True)
dfstr1.index_oper = dfstr1.index_oper.fillna('102976')

dfstr.index_oper = dfstr.index_oper.astype(float)
dfstr1.index_oper = dfstr1.index_oper.astype(int)
df = pd.concat([dfstr, dfstr1], axis=0)

df.index_oper = df.index_oper.astype(int)

df = df.drop(['mailrank',
              'postmark',
              'priority',
              'mailtype',
              'directctg',
              'name_mfi',
              'oper_type + oper_attr'], axis=1)

cat_cols = ['type',
            'is_privatecategory',
            'is_in_yandex',
            'is_return',]

target_col = df.label

In [80]:
test.index_oper = test.index_oper.astype(str)

teststr = test.loc[test.index_oper.str.slice(-2,) == '.0']
teststr1 = test.loc[(test.index_oper.str.slice(-2,) != '.0')]

teststr1.index_oper.replace(' ', np.nan, inplace=True)
teststr1.index_oper = teststr1.index_oper.fillna('102976')

teststr.index_oper = teststr.index_oper.astype(float)
teststr1.index_oper = teststr1.index_oper.astype(int)
test = pd.concat([teststr, teststr1], axis=0)

test.index_oper = test.index_oper.astype(int)

test = test.drop(['mailrank',
                  'postmark',
                  'priority',
                  'mailtype',
                  'directctg',
                  'name_mfi',
                  'oper_type + oper_attr'], axis=1)

In [82]:
df.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

,id,index_oper,class,weight,mailctg,transport_pay,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
id,1.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00
index_oper,0.00,1.00,0.23,0.04,0.08,0.07,-0.00,-0.02,-0.48,-0.60,-0.36,-0.50,-0.52,0.02,0.09,-0.10,0.01,-0.03
class,-0.00,0.23,1.00,0.00,0.01,0.00,-0.01,-0.01,-0.28,-0.35,-0.20,-0.28,-0.34,0.02,0.04,0.00,0.02,0.12
weight,0.00,0.04,0.00,1.00,0.22,0.59,0.35,0.10,-0.06,-0.05,-0.05,-0.06,-0.04,-0.02,0.11,-0.14,-0.02,-0.04
mailctg,-0.00,0.08,0.01,0.22,1.00,0.27,0.18,0.08,-0.09,-0.10,-0.07,-0.09,-0.09,-0.07,0.08,-0.56,-0.06,-0.15
transport_pay,-0.00,0.07,0.00,0.59,0.27,1.00,0.17,-0.02,-0.08,-0.09,-0.05,-0.08,-0.06,-0.03,0.16,-0.17,-0.02,-0.06
weight_mfi,0.00,-0.00,-0.01,0.35,0.18,0.17,1.00,0.41,-0.06,0.00,-0.06,-0.04,-0.04,-0.03,-0.10,-0.13,-0.02,-0.03
price_mfi,0.00,-0.02,-0.01,0.10,0.08,-0.02,0.41,1.00,-0.01,0.02,-0.02,-0.01,-0.01,-0.01,-0.05,-0.05,-0.01,-0.01
dist_qty_oper_login_1,0.00,-0.48,-0.28,-0.06,-0.09,-0.08,-0.06,-0.01,1.00,0.69,0.84,0.89,0.80,-0.01,-0.05,0.12,0.00,0.13
total_qty_oper_login_1,0.00,-0.60,-0.35,-0.05,-0.10,-0.09,0.00,0.02,0.69,1.00,0.53,0.78,0.71,-0.01,-0.10,0.11,0.00,0.06


In [81]:
df.sample(10)

,id,index_oper,type,is_privatecategory,class,is_in_yandex,is_return,weight,mailctg,transport_pay,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
2174394,2819512,102976,ММПО,N,0.0,N,N,180.0,1.0,24.29,138.0,104.0,1089.0,64270133.0,116432632.0,180702765.0,188407812.0,0,0,0,0,0
5113255,3091244,102976,ММПО,N,0.0,N,N,19.0,0.0,0.00,17.0,40.0,910.0,55549307.0,88774964.0,144324271.0,149073880.0,0,0,1,0,0
2283809,2455768,102971,Цех,N,0.0,N,N,625.0,1.0,0.00,308.0,300.0,172.0,48772076.0,11990.0,48784066.0,58042096.0,0,0,0,0,0
874360,238300,102976,ММПО,N,0.0,N,N,494.0,1.0,34.36,0.0,0.0,1142.0,60526245.0,96079047.0,156605292.0,163843348.0,0,0,0,0,0
396706,2403444,111950,Цех,N,0.0,Y,N,57.0,1.0,0.00,55.0,941.0,113.0,7258279.0,41587.0,7299866.0,132348119.0,0,0,0,0,0
2384283,1951394,302960,МСЦ,N,0.0,Y,N,420.0,1.0,0.00,0.0,0.0,46.0,709903.0,311147.0,1021050.0,11364791.0,0,1,0,0,0
5875897,2491050,443962,Цех,N,0.0,Y,N,101.0,1.0,0.00,100.0,2976.0,54.0,2945559.0,192.0,2945751.0,54220629.0,0,0,0,0,0
3638346,5862171,102976,ММПО,N,0.0,N,N,25.0,0.0,0.00,50.0,100.0,1089.0,64270133.0,116432632.0,180702765.0,188407812.0,0,1,1,0,0
4645021,1642981,610962,Участок,N,0.0,Y,N,50.0,1.0,0.00,49.0,115.0,36.0,1027800.0,953650.0,1981450.0,14325807.0,0,1,0,0,0
4409424,5634494,102976,ММПО,N,0.0,N,N,16.0,1.0,0.00,30.0,277.0,972.0,68766046.0,91123247.0,159889293.0,164927295.0,0,0,0,0,0


In [83]:
target_col.value_counts()

0    5829020
1     170980
Name: label, dtype: int64

In [84]:
weight = (target_col[target_col == 0].count()) / (target_col[target_col == 1].count())
weight

34.09182360510001

In [ ]:
df = df.drop('label', axis=1)

In [85]:
cat = CatBoostClassifier(cat_features=cat_cols,
                         verbose=True,
                         scale_pos_weight=weight,
                         learning_rate = 0.02,
                         l2_leaf_reg = 2,
                         depth = 8,
                         iterations=1500)

In [58]:
cat.fit(df, target_col)

0:	learn: 0.6715114	total: 4.67s	remaining: 1h 56m 34s
1:	learn: 0.6520441	total: 9.68s	remaining: 2h 50s
2:	learn: 0.6302591	total: 14.3s	remaining: 1h 58m 39s
3:	learn: 0.6105337	total: 18.8s	remaining: 1h 56m 56s
4:	learn: 0.5909069	total: 22.6s	remaining: 1h 52m 29s
5:	learn: 0.5749370	total: 27s	remaining: 1h 51m 55s
6:	learn: 0.5605785	total: 31.2s	remaining: 1h 50m 44s
7:	learn: 0.5452537	total: 35.2s	remaining: 1h 49m 17s
8:	learn: 0.5314614	total: 39.3s	remaining: 1h 48m 32s
9:	learn: 0.5181883	total: 43.5s	remaining: 1h 47m 56s
10:	learn: 0.5047330	total: 47.4s	remaining: 1h 46m 56s
11:	learn: 0.4926103	total: 51.2s	remaining: 1h 45m 48s
12:	learn: 0.4821577	total: 55.5s	remaining: 1h 45m 45s
13:	learn: 0.4724447	total: 59.4s	remaining: 1h 45m 7s
14:	learn: 0.4632518	total: 1m 3s	remaining: 1h 45m 11s
15:	learn: 0.4550333	total: 1m 7s	remaining: 1h 44m 47s
16:	learn: 0.4464355	total: 1m 12s	remaining: 1h 44m 43s
17:	learn: 0.4383771	total: 1m 15s	remaining: 1h 43m 52s
18:	lea

In [ ]:
prediction = cat.predict(test)

In [ ]:
sub = pd.DataFrame(test['id'])
sub['label'] = prediction
sub.to_csv('ans.csv', sep=",", index=False, line_terminator='\n')